In [ ]:
# Dans ce notebook, nous avons testé le modèle de facebook : Prophet pour prédire l'évolution des imports de vêtements usagés en France pour les prochaines années.

In [ ]:
#import des librairies utiles pour la prédiction

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from prophet import Prophet

# définition des warnings à ignorer
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=UserWarning)


In [ ]:
#import de  la table nécessaire depuis BigQuery

from google.colab import auth
auth.authenticate_user()

query = "SELECT * FROM projet-fripes.Data_cleaning_en_cours.onu_import_france_mois"
df = pd.read_gbq(query, project_id = "pragmatic-cache-475010-p8")
df.head()

In [ ]:
# Mise en forme du dataframe

from datetime import date
df = df.rename(columns={'date_date':'ds', 'poids': 'y'})

In [ ]:
# séparation des données en deux sections : train et test

train = df[df['ds'] < pd.Timestamp('2023-01-01')]
test = df[df['ds'] >= pd.Timestamp('2023-01-01')]

In [ ]:
# vérification de la lon,gueur des sections train et test

print(f"Number of months in train data: {len(train)}")
print(f"Number of months in test data: {len(test)}")

In [ ]:
#instanciation du modèle

m = Prophet()
m.fit(train)

In [ ]:
future = m.make_future_dataframe(periods=150, freq='M')
future.tail()

In [ ]:
# prédiction

forecast = m.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(20)

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
fig2 = m.plot_components(forecast)

In [ ]:
# présentation de la prédiction sur un graphique

from prophet.plot import plot_plotly, plot_components_plotly

plot_plotly(m, forecast)

In [ ]:
# calcul de l'écart quadratique moyen (RMSE)

from statsmodels.tools.eval_measures import rmse

predictions = forecast.iloc[-len(test):]['yhat']
actuals = test['y']

print(f"RMSE: {round(rmse(predictions, actuals))}")

In [ ]:
# validation croisée

from prophet.diagnostics import cross_validation
df_cv = cross_validation(m, initial='730 days', period='180 days', horizon = '365 days')


In [ ]:
# résultats de la validation croisée en tableau

df_cv.head()

In [ ]:
# indicateurs de performance du modèle

from prophet.diagnostics import performance_metrics
df_p = performance_metrics(df_cv)
df_p.head(20)